In [ ]:
!git clone https://github.com/UCSD-AI4H/COVID-CT.git

Cloning into 'COVID-CT'...
remote: Enumerating objects: 5463, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5463 (delta 0), reused 1 (delta 0), pack-reused 5459
Receiving objects: 100% (5463/5463), 1.09 GiB | 4.02 MiB/s, done.
Resolving deltas: 100% (360/360), done.
Checking out files: 100% (1048/1048), done.


In [ ]:
## unzip archives  q - quite, n - don't overwrite
!unzip -q -n '/content/COVID-CT/Images-processed/CT_COVID.zip'
!unzip -q -n '/content/COVID-CT/Images-processed/CT_NonCOVID.zip'

In [ ]:
import os
path='/content/CT_COVID/'
for count, filename in enumerate(os.listdir(path)): 
    dst ="covid-" + str(count) + ".png"
    src =path+ filename 
    dst =path+ dst 
        
    # rename() function will 
    # rename all the files 
    os.rename(src, dst)
print("Covid Positive cases: ", count)

path='/content/CT_NonCOVID/'
for count, filename in enumerate(os.listdir(path)): 
    dst ="noncovid-" + str(count) + ".png"
    src =path+ filename 
    dst =path+ dst 
        
    # rename() function will 
    # rename all the files 
    os.rename(src, dst)

print("Covid Negative cases: ", count)

Covid Positive cases:  348
Covid Negative cases:  396


## Exercise
We have downlaoded the COVID Image Data for you. Now, please a train and build a classification model to detect COVID vs Non-Covid cases. 

In [ ]:
# Make new base directory
positive_dataset_dir = '/content/CT_COVID'
negative_dataset_dir = '/content/CT_NonCOVID'
base_dir = '/content/COVID_small'
os.mkdir(base_dir)

In [ ]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

train_pos_dir = os.path.join(train_dir, 'positive')
os.mkdir(train_pos_dir)

train_neg_dir = os.path.join(train_dir, 'negative')
os.mkdir(train_neg_dir)

validation_pos_dir = os.path.join(validation_dir, 'positive')
os.mkdir(validation_pos_dir)

validation_neg_dir = os.path.join(validation_dir, 'negative')
os.mkdir(validation_neg_dir)

test_pos_dir = os.path.join(test_dir, 'positive')
os.mkdir(test_pos_dir)

test_neg_dir = os.path.join(test_dir, 'negative')
os.mkdir(test_neg_dir)

In [ ]:
import shutil
fnames = ['covid-{}.png'.format(i) for i in range(250)]
for fname in fnames:
    src = os.path.join(positive_dataset_dir, fname)
    dst = os.path.join(train_pos_dir, fname)
    #print(src,dst)
    shutil.copyfile(src, dst)

fnames = ['covid-{}.png'.format(i) for i in range(250, 300)]
for fname in fnames:
    src = os.path.join(positive_dataset_dir, fname)
    dst = os.path.join(validation_pos_dir, fname)
    #print(src,dst)
    shutil.copyfile(src, dst)

fnames = ['covid-{}.png'.format(i) for i in range(300, 348)]
for fname in fnames:
    src = os.path.join(positive_dataset_dir, fname)
    dst = os.path.join(test_pos_dir, fname)
    #print(src,dst)
    shutil.copyfile(src, dst)    

fnames = ['noncovid-{}.png'.format(i) for i in range(300)]
for fname in fnames:
    src = os.path.join(negative_dataset_dir, fname)
    dst = os.path.join(train_neg_dir, fname)
    #print(src,dst)
    shutil.copyfile(src, dst)

fnames = ['noncovid-{}.png'.format(i) for i in range(300, 350)]
for fname in fnames:
    src = os.path.join(negative_dataset_dir, fname)
    dst = os.path.join(validation_neg_dir, fname)
    #print(src,dst)
    shutil.copyfile(src, dst)

fnames = ['noncovid-{}.png'.format(i) for i in range(350, 396)]
for fname in fnames:
    src = os.path.join(negative_dataset_dir, fname)
    dst = os.path.join(test_neg_dir, fname)
    #print(src,dst)
    shutil.copyfile(src, dst)

In [ ]:
print('total training positive images:', len(os.listdir(train_pos_dir)))
print('total training negative images:', len(os.listdir(train_neg_dir)))

print('total validation positive images:', len(os.listdir(validation_pos_dir)))
print('total validation negative images:', len(os.listdir(validation_neg_dir)))

print('total test positive images:', len(os.listdir(test_pos_dir)))
print('total test negative images:', len(os.listdir(test_neg_dir)))

total training positive images: 250
total training negative images: 300
total validation positive images: 50
total validation negative images: 50
total test positive images: 48
total test negative images: 46


In [ ]:
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
#model.add(layers.Dropout(0.5)) # Dropout
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150), 
                                                    batch_size=10,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=15,
                                                        class_mode='binary')
history = model.fit_generator(train_generator,
                              steps_per_epoch=30,
                              epochs=7,
                              validation_data=validation_generator,
                              validation_steps=5)

Found 550 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Epoch 1/7
 1/30 [>.............................] - ETA: 1s - loss: 0.5328 - acc: 0.9000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


30/30 [==============================] - 2s 63ms/step - loss: 0.3432 - acc: 0.8467 - val_loss: 0.5417 - val_acc: 0.7467
Epoch 2/7
30/30 [==============================] - 2s 57ms/step - loss: 0.3506 - acc: 0.8633 - val_loss: 0.5780 - val_acc: 0.7467
Epoch 3/7
30/30 [==============================] - 2s 63ms/step - loss: 0.3351 - acc: 0.8700 - val_loss: 0.4779 - val_acc: 0.7600
Epoch 4/7
30/30 [==============================] - 2s 66ms/step - loss: 0.2784 - acc: 0.8933 - val_loss: 0.5652 - val_acc: 0.7467
Epoch 5/7
30/30 [==============================] - 2s 67ms/step - loss: 0.2945 - acc: 0.8800 - val_loss: 0.5659 - val_acc: 0.7733
Epoch 6/7
30/30 [==============================] - 2s 60ms/step - loss: 0.2935 - acc: 0.9000 - val_loss: 0.5235 - val_acc: 0.8133
Epoch 7/7
30/30 [==============================] - 2s 65ms/step - loss: 0.2526 - acc: 0.9067 - val_loss: 0.5871 - val_acc: 0.7467


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150), 
                                                    batch_size=10,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')

history = model.fit_generator(train_generator,
                              steps_per_epoch=30,
                              epochs=7,
                              validation_data=validation_generator,
                              validation_steps=5)

Found 550 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Epoch 1/7
 1/30 [>.............................] - ETA: 1s - loss: 0.4797 - acc: 0.7000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


30/30 [==============================] - 2s 70ms/step - loss: 0.6473 - acc: 0.6133 - val_loss: 0.6634 - val_acc: 0.5800
Epoch 2/7
30/30 [==============================] - 2s 67ms/step - loss: 0.6433 - acc: 0.6733 - val_loss: 0.6670 - val_acc: 0.5700
Epoch 3/7
30/30 [==============================] - 2s 62ms/step - loss: 0.6284 - acc: 0.6200 - val_loss: 0.6460 - val_acc: 0.6500
Epoch 4/7
30/30 [==============================] - 2s 64ms/step - loss: 0.6034 - acc: 0.6767 - val_loss: 0.6287 - val_acc: 0.5800
Epoch 5/7
30/30 [==============================] - 2s 67ms/step - loss: 0.5832 - acc: 0.6767 - val_loss: 0.6082 - val_acc: 0.6400
Epoch 6/7
30/30 [==============================] - 2s 62ms/step - loss: 0.5498 - acc: 0.7233 - val_loss: 0.5964 - val_acc: 0.6600
Epoch 7/7
30/30 [==============================] - 2s 63ms/step - loss: 0.5663 - acc: 0.7067 - val_loss: 0.5812 - val_acc: 0.6800
